In [1]:
# import relevant libraries
import csv_diff
import pandas as pd
import datetime
import sys
import csv

In [2]:
# change limits on CSV column size
csv.field_size_limit(sys.maxsize)

131072

In [3]:
# load current and previous release data
data_previous_release = csv_diff.load_csv(open('data/21_11_data_v1.csv'), key='ensembl_gene_id')
data_current_release = csv_diff.load_csv(open('data/22_02_data_v1.csv'), key='ensembl_gene_id') 

In [4]:
# calculate number of records in each release data file and difference
difference_in_release_data = len(data_current_release) - len(data_previous_release) 
print("Records in the previous release data = %i" % len(data_previous_release))
print("Records in the current release data = %i" % len(data_current_release))
print("Difference between the number of rows in the two release data files = %i" % (difference_in_release_data))

Records in the previous release data = 19498
Records in the current release data = 19475
Difference between the number of rows in the two release data files = -23


In [5]:
# run `csv_diff` library on data files and generate JSON output
all_csv_diff_results = csv_diff.compare(data_previous_release, data_current_release)

In [6]:
# normalise JSON generated by `csv_diff` library and filter for entries where data has changed between releases
normalised_all_csv_diff_results = pd.json_normalize(all_csv_diff_results['changed'])

In [7]:
# convert normalised JSON into a data frame
all_tractability_changes_df = pd.DataFrame.from_dict(normalised_all_csv_diff_results, orient='columns')

In [8]:
# print number of records (Ensembl IDs) where data has changed between releases
print('Generated tractability diff file with %i records changed' % (len(all_tractability_changes_df)))

Generated tractability diff file with 784 records changed


In [9]:
# generate and save ALL_COLUMNS report, which shows differences in all columns between release data files
today_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_directory = 'reports/'
report_filename = today_date + '_' + 'tractability_data_comparison_ALL_COLUMNS_report.csv'
all_tractability_changes_df.to_csv(report_directory + report_filename, index=False)

In [10]:
# list columns in ALL_COLUMNS report where data has changed
all_tractability_changes_df.columns

Index(['key', 'changes.AB_B9_Human Protein Atlas loc',
       'changes.PR_B5_UniProt Ubiquitination',
       'changes.SM_B4_Structure with Ligand', 'changes.AB_B8_GO CC med conf',
       'changes.AB_B5_GO CC high conf', 'changes.AB_B4_UniProt loc high conf',
       'changes.AB_B6_UniProt loc med conf', 'changes.accession',
       'changes.AB_B7_UniProt SigP or TMHMM', 'changes.symbol',
       'changes.PR_B7_Half-life Data', 'changes.SM_B1_Approved Drug',
       'changes.SM_B5_High-Quality Ligand',
       'changes.PR_B8_Small Molecule Binder',
       'changes.PR_B6_Database Ubiquitination'],
      dtype='object')

In [11]:
# set list of all clinical precedence buckets needed to generate subset report (CLIN_PRECEDENCE) that is used
# to manually check if new release tractability data is in sync with previous release tractability data
# and ChEMBL evidence strings
all_clinical_precedence_buckets = [
    'changes.SM_B1_Approved Drug',
    'changes.SM_B2_Advanced Clinical',
    'changes.SM_B3_Phase 1 Clinical',
    'changes.AB_B1_Approved Drug',
    'changes.AB_B2_Advanced Clinical',
    'changes.AB_B3_Phase 1 Clinical',
    'changes.OC_B1_Approved Drug',
    'changes.OC_B2_Advanced Clinical',
    'changes.OC_B3_Phase 1 Clinical',
    'changes.PR_B1_Approved Drug'
    'changes.PR_B2_Advanced Clinical',
    'changes.PR_B3_Phase 1 Clinical'
]

In [12]:
# create new list of columns to filter ALL_COLUMNS report 
# and start with `key` field that contains Ensembl IDs
columns_for_CLIN_PRECEDENCE_report = ['key'] 

In [13]:
# check if each clinical precedence bucket label is in the list of columns where data changed 
# in the ALL_COLUMNS report -- if the bucket label is in the ALL_COLUMNS report column list, 
# add to a new list that will be used to generate the CLIN_PRECEDENCE report
for bucket in all_clinical_precedence_buckets:
    if bucket in all_tractability_changes_df.columns:
        columns_for_CLIN_PRECEDENCE_report.append(bucket)

In [14]:
# print list of clinical precedence buckets found in ALL_COLUMNS report
print(columns_for_CLIN_PRECEDENCE_report)

['key', 'changes.SM_B1_Approved Drug']


In [15]:
# generate new data frame with clinical precedence buckets columns where data has changed
tractability_changes_clin_precedence_only_df = all_tractability_changes_df.filter(columns_for_CLIN_PRECEDENCE_report)

In [16]:
# set index of new data frame to be Ensembl ID in `key` column
tractability_changes_clin_precedence_only_df.set_index('key', inplace=True, drop=True)

In [17]:
# drop rows where all cells are NaN or null
tractability_changes_clin_precedence_only_df.dropna(how="all", inplace=True)

In [18]:
# print number of Ensembl IDs that need to be manually reviewed
print('Number of Ensembl IDs to be manually reviewed: %i' % (len(tractability_changes_clin_precedence_only_df)))

Number of Ensembl IDs to be manually reviewed: 1


In [19]:
# generate new report of changes in clinical precedence bucket
# note: this is the report that you can use to manually check the new tractability assessments
# against the previous release tractability assessments and the ChEMBL evidence strings
today_date = datetime.datetime.now().strftime('%Y-%m-%d')
report_directory = 'reports/'
report_filename = today_date + '_' + 'tractability_data_comparison_CLIN_PRECEDENCE_report.csv'
tractability_changes_clin_precedence_only_df.to_csv(report_directory + report_filename, index=True)